In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    BBB_Train_Decile9 = pd.read_pickle("BBB_Train_Decile9.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile9.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile9, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 9')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 17s 5ms/step - loss: 0.3153 - acc: 0.8957 - val_loss: 0.2956 - val_acc: 0.8951
Epoch 48/75
3087/3087 [==============================] - 17s 5ms/step - loss: 0.3079 - acc: 0.8963 - val_loss: 0.2944 - val_acc: 0.8951
Epoch 49/75
3087/3087 [==============================] - 17s 5ms/step - loss: 0.3159 - acc: 0.8947 - val_loss: 0.2945 - val_acc: 0.8990
Epoch 50/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3125 - acc: 0.8937 - val_loss: 0.2928 - val_acc: 0.8977
Epoch 51/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3139 - acc: 0.8899 - val_loss: 0.2923 - val_acc: 0.8990
Epoch 52/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3134 - acc: 0.8931 - val_loss: 0.2949 - val_acc: 0.8964
Epoch 53/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.3108 - acc: 0.8915 - val_loss: 0.2909 - val_acc: 0.8964
Epoch 54/75
3087/3087 [==============================] - 14s

3087/3087 [==============================] - 17s 5ms/step - loss: 0.2787 - acc: 0.8980 - val_loss: 0.2730 - val_acc: 0.9028
Epoch 31/75
3087/3087 [==============================] - 17s 5ms/step - loss: 0.2723 - acc: 0.9018 - val_loss: 0.2729 - val_acc: 0.9016

Epoch 00031: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 32/75
3087/3087 [==============================] - 17s 5ms/step - loss: 0.2691 - acc: 0.9022 - val_loss: 0.2729 - val_acc: 0.9016
Epoch 33/75
3087/3087 [==============================] - 17s 5ms/step - loss: 0.2744 - acc: 0.9048 - val_loss: 0.2728 - val_acc: 0.9028
Epoch 34/75
3087/3087 [==============================] - 16s 5ms/step - loss: 0.2742 - acc: 0.9009 - val_loss: 0.2727 - val_acc: 0.9028
Epoch 35/75
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2711 - acc: 0.9031 - val_loss: 0.2726 - val_acc: 0.9028
Epoch 36/75
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2757 - acc: 0.9025 - val_loss: 0.2728 - val_acc: 0.

3087/3087 [==============================] - 16s 5ms/step - loss: 0.3222 - acc: 0.8973 - val_loss: 0.2955 - val_acc: 0.8977
Epoch 15/100
3087/3087 [==============================] - 18s 6ms/step - loss: 0.3169 - acc: 0.8902 - val_loss: 0.2925 - val_acc: 0.8977
Epoch 16/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.3151 - acc: 0.8905 - val_loss: 0.2963 - val_acc: 0.9003
Epoch 17/100
3087/3087 [==============================] - 18s 6ms/step - loss: 0.3103 - acc: 0.8921 - val_loss: 0.2921 - val_acc: 0.9003
Epoch 18/100
3087/3087 [==============================] - 18s 6ms/step - loss: 0.3224 - acc: 0.8899 - val_loss: 0.2857 - val_acc: 0.9016
Epoch 19/100
3087/3087 [==============================] - 18s 6ms/step - loss: 0.2995 - acc: 0.8970 - val_loss: 0.2873 - val_acc: 0.9003
Epoch 20/100
3087/3087 [==============================] - 18s 6ms/step - loss: 0.2929 - acc: 0.8963 - val_loss: 0.2857 - val_acc: 0.9003
Epoch 21/100
3087/3087 [==============================

Epoch 73/100
3087/3087 [==============================] - 16s 5ms/step - loss: 0.2775 - acc: 0.9002 - val_loss: 0.2713 - val_acc: 0.8951
Epoch 74/100
3087/3087 [==============================] - 17s 5ms/step - loss: 0.2749 - acc: 0.9009 - val_loss: 0.2714 - val_acc: 0.8951
Epoch 75/100
3087/3087 [==============================] - 17s 5ms/step - loss: 0.2751 - acc: 0.8973 - val_loss: 0.2713 - val_acc: 0.8951
Epoch 76/100
3087/3087 [==============================] - 17s 5ms/step - loss: 0.2807 - acc: 0.9009 - val_loss: 0.2714 - val_acc: 0.8964
Epoch 77/100
3087/3087 [==============================] - 16s 5ms/step - loss: 0.2853 - acc: 0.8970 - val_loss: 0.2714 - val_acc: 0.8938
Epoch 78/100
3087/3087 [==============================] - 17s 5ms/step - loss: 0.2718 - acc: 0.8999 - val_loss: 0.2714 - val_acc: 0.8938
Epoch 79/100
3087/3087 [==============================] - 17s 5ms/step - loss: 0.2758 - acc: 0.9006 - val_loss: 0.2714 - val_acc: 0.8938
Epoch 80/100
3087/3087 [=================

...

3087/3087 [==============================] - 19s 6ms/step - loss: 0.4356 - acc: 0.8128 - val_loss: 0.4128 - val_acc: 0.8122
Epoch 17/125
3087/3087 [==============================] - 18s 6ms/step - loss: 0.4255 - acc: 0.8228 - val_loss: 0.4158 - val_acc: 0.8057
Epoch 18/125
3087/3087 [==============================] - 19s 6ms/step - loss: 0.4254 - acc: 0.8209 - val_loss: 0.4056 - val_acc: 0.8174
Epoch 19/125
3087/3087 [==============================] - 18s 6ms/step - loss: 0.4161 - acc: 0.8358 - val_loss: 0.3998 - val_acc: 0.8251
Epoch 20/125
3087/3087 [==============================] - 19s 6ms/step - loss: 0.4142 - acc: 0.8309 - val_loss: 0.3966 - val_acc: 0.8277
Epoch 21/125
3087/3087 [==============================] - 17s 6ms/step - loss: 0.4137 - acc: 0.8374 - val_loss: 0.3940 - val_acc: 0.8290
Epoch 22/125
3087/3087 [==============================] - 17s 6ms/step - loss: 0.4052 - acc: 0.8367 - val_loss: 0.3893 - val_acc: 0.8329
Epoch 23/125
3087/3087 [==============================


Epoch 00089: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 90/125
3087/3087 [==============================] - 11s 4ms/step - loss: 0.3087 - acc: 0.8905 - val_loss: 0.2886 - val_acc: 0.8951
Epoch 91/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3089 - acc: 0.8892 - val_loss: 0.2882 - val_acc: 0.8964
Epoch 92/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3112 - acc: 0.8950 - val_loss: 0.2877 - val_acc: 0.8951
Epoch 93/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3016 - acc: 0.8941 - val_loss: 0.2878 - val_acc: 0.8951
Epoch 94/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3079 - acc: 0.8970 - val_loss: 0.2875 - val_acc: 0.8977
Epoch 95/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3102 - acc: 0.8908 - val_loss: 0.2874 - val_acc: 0.8977
Epoch 96/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3058 - acc: 0.8947 - val_loss: 0

3087/3087 [==============================] - 11s 4ms/step - loss: 0.2825 - acc: 0.8967 - val_loss: 0.2686 - val_acc: 0.8977
Epoch 24/100
3087/3087 [==============================] - 11s 4ms/step - loss: 0.2785 - acc: 0.8983 - val_loss: 0.2687 - val_acc: 0.8990
Epoch 25/100
3087/3087 [==============================] - 11s 4ms/step - loss: 0.2815 - acc: 0.8941 - val_loss: 0.2692 - val_acc: 0.8977
Epoch 26/100
3087/3087 [==============================] - 11s 4ms/step - loss: 0.2825 - acc: 0.8980 - val_loss: 0.2689 - val_acc: 0.8990
Epoch 27/100
3087/3087 [==============================] - 11s 4ms/step - loss: 0.2668 - acc: 0.9031 - val_loss: 0.2690 - val_acc: 0.8977
Epoch 28/100
3087/3087 [==============================] - 11s 4ms/step - loss: 0.2735 - acc: 0.9012 - val_loss: 0.2685 - val_acc: 0.8977

Epoch 00028: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 29/100
3087/3087 [==============================] - 11s 4ms/step - loss: 0.2743 - acc: 0.9006 - val_los

Epoch 82/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2764 - acc: 0.8989 - val_loss: 0.2686 - val_acc: 0.8977
Epoch 83/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2708 - acc: 0.9038 - val_loss: 0.2687 - val_acc: 0.8977
Epoch 84/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2673 - acc: 0.9054 - val_loss: 0.2689 - val_acc: 0.8964
Epoch 85/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2753 - acc: 0.8989 - val_loss: 0.2689 - val_acc: 0.8951
Epoch 86/100
3087/3087 [==============================] - 8s 2ms/step - loss: 0.2756 - acc: 0.9006 - val_loss: 0.2688 - val_acc: 0.8964
Epoch 87/100
3087/3087 [==============================] - 8s 2ms/step - loss: 0.2781 - acc: 0.8980 - val_loss: 0.2688 - val_acc: 0.8964
Epoch 88/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.2663 - acc: 0.9018 - val_loss: 0.2687 - val_acc: 0.8977
Epoch 89/100
3087/3087 [========================

Epoch 41/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2848 - acc: 0.9002 - val_loss: 0.2765 - val_acc: 0.8977
Epoch 42/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2811 - acc: 0.9015 - val_loss: 0.2763 - val_acc: 0.8990
Epoch 43/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2787 - acc: 0.8960 - val_loss: 0.2763 - val_acc: 0.8990
Epoch 44/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2867 - acc: 0.8986 - val_loss: 0.2764 - val_acc: 0.8990
Epoch 45/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2801 - acc: 0.8976 - val_loss: 0.2762 - val_acc: 0.8990
Epoch 46/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2848 - acc: 0.9006 - val_loss: 0.2763 - val_acc: 0.8990
Epoch 47/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2809 - acc: 0.8996 - val_loss: 0.2762 - val_acc: 0.8990
Epoch 48/50
3087/3087 [==============================] 

In [ ]:
0.8297